In [102]:
import pysam
import numpy as np
import pandas as pd

Создаем dataframe

In [103]:
data = pd.read_csv("FP_SNPs.txt", sep='\t')
data.head()

,rs#,chromosome,GB37_position,GB38_position,allele1,allele2
0,2887286,1,1156131,1220751,C,T
1,6685064,1,1211292,1275912,T,C
2,2840528,1,2283896,2352457,A,G
3,3890745,1,2553624,2622185,C,T
4,1798246,1,3080855,3164291,C,A


Преобразование файла FP_snps.txt к формату похожему по
на VCF:

In [104]:
data= data[data['chromosome']!=23]
data = data.drop('GB37_position', axis=1)
data['chromosome'] = data['chromosome'].astype(str)
data['rs#'] = data['rs#'].astype(str)
data['CHROM'] =  'chr'+data['chromosome']
data = data.drop('chromosome', axis=1)
data['ID'] =  'rs'+data['rs#']
data = data.drop('rs#', axis=1)
data = data.rename(columns={'GB38_position': 'POS'})
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   POS      10000 non-null  int64 
 1   allele1  10000 non-null  object
 2   allele2  10000 non-null  object
 3   CHROM    10000 non-null  object
 4   ID       10000 non-null  object
dtypes: int64(1), object(4)
memory usage: 468.8+ KB


Создаем fastafile

In [105]:
fastafile = pysam.FastaFile('GRCh38.d1.vd1.fa')

Создаем столбец REF

In [106]:
data['REF'] = np.zeros(len(data))
for i in range(len(data)):
    data['REF'][i] = fastafile.fetch(data['CHROM'][i], data['POS'][i], data['POS'][i] +1)

/var/folders/d_/y4m68ky157n56tm5fxmw5mth0000gn/T/ipykernel_56364/949429242.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['REF'][i] = fastafile.fetch(data['CHROM'][i], data['POS'][i], data['POS'][i] +1)


Сравниваем аллели с референсными занчениями

In [107]:
data['ALT'] = np.zeros(len(data))
for i in range(len(data)):
    if (data['REF'][i] == data['allele1'][i]) and (data['REF'][i] == data['allele2'][i]):
        data['ALT'][i] = '1/1'
    elif (data['REF'][i] != data['allele1'][i]) and (data['REF'][i] != data['allele2'][i]):
        data['ALT'][i] = '0/0'
    elif (data['REF'][i] != data['allele1'][i]) and (data['REF'][i] == data['allele2'][i]):
        data['ALT'][i] = '0/1'
    else:
        data['ALT'][i] = '1/0'

/var/folders/d_/y4m68ky157n56tm5fxmw5mth0000gn/T/ipykernel_56364/3924278200.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ALT'][i] = '0/0'


In [108]:
data = data.reindex(columns=['CHROM', 'POS', 'ID', 'REF', 'ALT', 'allele1', 'allele2'])

Итоговая таблица

In [109]:
data.head(5)

,CHROM,POS,ID,REF,ALT,allele1,allele2
0,chr1,1220751,rs2887286,A,0/0,C,T
1,chr1,1275912,rs6685064,A,0/0,T,C
2,chr1,2352457,rs2840528,G,0/1,A,G
3,chr1,2622185,rs3890745,G,0/0,C,T
4,chr1,3164291,rs1798246,A,0/1,C,A
